# RFormula:

- Linguagem R permite definir modelo através de fórmula com o padrão: <br>
    var dependente ~ vars independentes
- variáveis independentes podem ser definidas através de "+" (qdo houver mais de uma)
- "." define todos os atributos, menos variável dependente
- Spark implementa Rformula (Combina vars independentes em uma únicao coluna)
    1. Colunas numéricas são transformadas em double
    2. Strings serão transformadas com String Indexer, e a última categoria é excluída e então aplica
    One Hot Encoding

Exemplo 1: HP ~ Consumo + Cilindros + Cilindradas
regressão para determinação de HP em função das dependentes ccc

Exemplo 2: HP ~ . 
regressão para determinação de HP em função de todas outras variáveis que são as independentes

In [1]:
#imports

import findspark, pyspark
from pyspark.sql import SparkSession
findspark.init() # inicializa interface pyspark jupyter
spark = SparkSession.builder.appName("RFormula").getOrCreate()
#inicializa sessão no spark com nome importação

In [2]:
from pyspark.ml.feature import RFormula

In [3]:
carros = spark.read.csv("Carros.csv", inferSchema = True, header=True,sep=";")
carros.show(5)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



In [7]:
Rformula= RFormula(formula = "HP ~ Consumo + Cilindros + Cilindradas",  featuresCol = "independente", labelCol="dependente")
carrosrf = Rformula.fit(carros).transform(carros)
carrosrf.select("independente","dependente").show()

+------------------+----------+
|      independente|dependente|
+------------------+----------+
|  [21.0,6.0,160.0]|     110.0|
|  [21.0,6.0,160.0]|     110.0|
| [228.0,4.0,108.0]|      93.0|
| [214.0,6.0,258.0]|     110.0|
| [187.0,8.0,360.0]|     175.0|
| [181.0,6.0,225.0]|     105.0|
| [143.0,8.0,360.0]|     245.0|
|[244.0,4.0,1467.0]|      62.0|
|[228.0,4.0,1408.0]|      95.0|
|[192.0,6.0,1676.0]|     123.0|
|[178.0,6.0,1676.0]|     123.0|
|[164.0,8.0,2758.0]|     180.0|
|[173.0,8.0,2758.0]|     180.0|
|[152.0,8.0,2758.0]|     180.0|
| [104.0,8.0,472.0]|     205.0|
| [104.0,8.0,460.0]|     215.0|
| [147.0,8.0,440.0]|     230.0|
| [324.0,4.0,787.0]|      66.0|
| [304.0,4.0,757.0]|      52.0|
| [339.0,4.0,711.0]|      65.0|
+------------------+----------+
only showing top 20 rows



In [8]:
carros.printSchema()

root
 |-- Consumo: integer (nullable = true)
 |-- Cilindros: integer (nullable = true)
 |-- Cilindradas: integer (nullable = true)
 |-- RelEixoTraseiro: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- Tempo: integer (nullable = true)
 |-- TipoMotor: integer (nullable = true)
 |-- Transmissao: integer (nullable = true)
 |-- Marchas: integer (nullable = true)
 |-- Carburadors: integer (nullable = true)
 |-- HP: integer (nullable = true)

